# Amazon Sagemaker Distributed Model Parallel을 사용하여 모델 병렬화로 BERT 후련 작업 시작

SageMaker 분산 모델 병렬 (SageMaker Distributed Model Parallel, SMP)은 GPU 메모리 제한으로 인해 이전에 학습하기 어려웠던 대규모 딥러닝 모델을 훈련하기 위한 모델 병렬 처리 라이브러리입니다. SageMaker Distributed Model Parallel은 여러 GPU 및 인스턴스에서 모델을 자동으로 효율적으로 분할하고 모델 훈련을 조정하므로 더 많은 매개 변수로 더 큰 모델을 생성하여 예측 정확도를 높일 수 있습니다.

이 노트북을 사용하여 PyTorch (버전 1.6.0) 및 [Amazon SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/overview.html#train-a-model-with-the-sagemaker-python-sdk)를 사용하여 모델을 훈련하도록 SMP를 구성합니다.

이 노트북에서는 SMP와 함께 BERT 예제 훈련 스크립트를 사용합니다.
예제 스크립트는 [Nvidia Deep Learning Examples](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/LanguageModeling/BERT)를 기반으로 하며, 아래 지침에 설명된 대로 데이터셋을 다운로드하여 Amazon Simple Storage Service(Amazon S3)에 업로드해야 합니다. 대용량 데이터셋이므로 연결 속도에 따라 이 프로세스를 완료하는 데 몇 시간이 걸릴 수 있습니다.


이 노트북은 아래 파일들이 필요합니다. Amazon SageMaker 예제 노트북 저장소의 모델 병렬 섹션에 있는 [bert directory](https://github.com/aws/amazon-sagemaker-examples/tree/master/training/distributed_training/pytorch/model_parallel/bert)에서 모든 파일들을 찾을 수 있습니다.


* `bert_example/sagemaker_smp_pretrain.py`: 노트북 지침에서 Pytorch estimator로 전달되는 엔트리 포인트 스크립트입니다. 이 스크립트는 SMP를 사용한 BERT 모델의 end-to-end 훈련을 담당합니다. 스크립트에는 SMP API가 사용되는 위치에 추가 주석이 있습니다.

* `bert_example/modeling.py`: BERT 모델에 대한 모델 정의가 포함됩니다.

* `bert_example/bert_config.json`: 모델의 추가 설정이 가능하며 `modeling.py`에서 사용됩니다. 추가 설정에는 드롭아웃 확률, 풀러(pooler) 및 인코더 크기, 인코더의 히든 레이어 수, 인코더의 중간 레이어 크기 등이 포함됩니다.

* `bert_example/schedulers.py`: BERT 모델 (`bert_example/sagemaker_smp_pretrain.py`) 의 end-to-end 훈련에 사용되는 학습률 스케줄러에 대한 정의를 포함합니다.

* `bert_example/utils.py`: BERT 모델 (`bert_example/sagemaker_smp_pretrain.py`) 의 end-to-end 훈ㄹㅕㄴ에 사용되는 다양한 헬퍼 유틸리티 함수를 포함합니다.

* `bert_example/file_utils.py`: 모델 정의에 사용되는 다양한 파일 유틸리티 함수 (`bert_example/modeling.py`) 를 포함합니다.

### 추가 리소스

Amazon SageMaker를 처음 사용하는 경우, SageMaker 상에서 SMP로 PyTorch 모델을 훈련 시 다음 정보들이 도움이 될 수 있습니다.

* SageMaker 모델 병렬 처리 라이브러리에 대한 자세한 내용은 [SageMaker Distributed를 사용한 모델 병렬 분산 훈련](http://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel.html)을 참조하세요.

* Pytorch와 함께 SageMaker Python SDK를 사용하는 방법에 대한 자세한 내용은 [SageMaker Python SDK와 함께 PyTorch 사용](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html)을 참조하세요.

* 자체 훈련 이미지로 Amazon SageMaker에서 훈련 작업을 시작하는 방법에 대한 자세한 내용은 [자체 훈련 알고리즘 사용](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html)을 참조하세요.


### Prerequisites 

1. 훈련에 사용할 입력 데이터를 저장하려면 S3 버켓을 생성해야 합니다. 이 버켓은 훈련 작업을 시작하는 데 사용하는 것과 동일한 AWS 리전에 있어야 합니다. 자세한 방법은 Amazon S3 문서의 [버켓 생성](https://docs.aws.amazon.com/AmazonS3/latest/gsg/CreatingABucket.html)을 참조하세요.

2. [Nvidia Deep Learning Examples](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/LanguageModeling/BERT)에서 훈련에 사용하는 데이터셋을 다운로드하고 생성한 S3 버켓에 업로드해야 합니다. 전처리 및 다운로드를 위해 제공되는 데이터셋 및 스크립트에 대한 자세한 내용은 Nvidia Deep Learning Examples repo README의 [데이터 가져 오기](https://github.com/NVIDIA/DeepLearningExamples/blob/master/PyTorch/LanguageModeling/BERT/README.md#getting-the-data)를 참조하세요. [빠른 시작 가이드](https://github.com/NVIDIA/DeepLearningExamples/blob/master/PyTorch/LanguageModeling/BERT/README.md#quick-start-guide)를 사용하여 데이터셋을 다운로드하는 방법을 익힐 수도 있습니다. 저장소는 세 개의 데이터셋으로 구성됩니다. 선택적으로 `wiki_only` 매개 변수를 사용하여 Wikipedia 데이터셋만 다운로드할 수 있습니다.

## Amazon SageMaker 초기화

Sagemaker SDK를 최신 버전으로 업그레이드하세요.

참고: 이 단계에서는 커널을 다시 시작해야 할 수 있습니다.

In [ ]:
import sagemaker
original_version = sagemaker.__version__
%pip install --upgrade sagemaker

노트북 인스턴스를 초기화합니다. AWS 리전, SageMaker 실행 역할 Amazon 리소스 이름 (ARN)을 가져옵니다.

In [ ]:
%%time
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
from sagemaker.pytorch import PyTorch
import boto3

role = get_execution_role() # provide a pre-existing role ARN as an alternative to creating a new role
print(f'SageMaker Execution Role:{role}')

client = boto3.client('sts')
account = client.get_caller_identity()['Account']
print(f'AWS account:{account}')

session = boto3.session.Session()
region = session.region_name
print(f'AWS region:{region}')
sagemaker_session = sagemaker.session.Session(boto_session=session)
import sys
print(sys.path)

## Amazon S3에서 훈련 데이터 준비 및 식별

S3 버켓에 BERT 데이터셋이 아직 없는 경우 [Nvidia BERT Example](https://github.com/NVIDIA/DeepLearningExamples/blob/master/PyTorch/LanguageModeling/BERT/README.md)의 지침을 참조하여 데이터셋을 다운로드하고 S3 버켓에 업로드하세요. 자세한 정보는 이 노트북의 시작 부분에 있는 Prerequisites을 참조하세요.

주석을 제거하고 다음 셀을 사용하여 훈련 데이터가 포함된 Amazon S3 버켓과 접두사(prefix)를 지정합니다. 예를 들어, g훈련 데이터가 s3://your-bucket/training 에 있는 경우 s3_bucket에` 'your-bucket'` 을 입력하고 접두사에 `'training'` 을 입력합니다. 출력 데이터는 `output/` 접두사 아래의 동일한 버킷에 저장됩니다.

In [ ]:
s3_bucket = '<ADD BUCKET>'
#prefix = '<ADD PREFIX>'

## SageMaker 데이터 채널 정의

이 단계에서는 Amazon SageMaker 훈련 데이터 채널과 출력 데이터 경로를 정의합니다. 훈련 데이터 채널은 S3에서 훈련 데이터의 위치를 식별합니다.

In [ ]:
s3train = f's3://{s3_bucket}/{prefix}'
train = sagemaker.session.TrainingInput(s3train, distribution='FullyReplicated', 
                                        s3_data_type='S3Prefix')

data_channels = {'train': train}

출력 데이터 경로를 설정합니다. 이 경로에 모델 아티팩트가 저장됩니다.

In [ ]:
s3_output_location = f's3://{s3_bucket}/output'
print(f'your output data will be stored in: s3://{s3_bucket}/output')

## SageMaker 훈련 작업 정의

다음으로 SageMaker Estimator API를 사용하여 SageMaker 훈련 작업을 정의합니다. [`PyTorchEstimator`](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/sagemaker.pytorch.html)를 사용하여 Amazon SageMaker가 훈련에 사용하는 EC2 인스턴스의 수와 유형을 정의합니다. 인스턴스에 연결된 볼륨의 크기도 포함됩니다.

Estimator 호출 시, 아래의 인자값들을 업데이트해야 합니다.
* `instance_count`
* `instance_type`
* `volume_size`

자세한 내용은 다음 하위 섹션을 참조하세요.

### 사용할 EC2 인스턴스의 유형 및 개수 업데이트

`instance_type` 및 `instance_count`에서 각각 지정하는 인스턴스 유형 및 인스턴스 수에 따라 Amazon SageMaker가 훈련 중에 사용하는 GPU 수가 결정됩니다. 명시 적으로`instance_type`은 단일 인스턴스의 GPU 수를 결정하고 그 숫자에 `instance_count`를 곱합니다.

훈련에 사용할 수 있는 총 GPU 수가 훈련 스크립트의 `smp.init`의 `config`에 있는 `partitions`와 같도록 `instance_type`및 `instance_count`의 값을 지정해야 합니다.

ddp를 `True`로 설정하는 경우, 사용 가능한 총 GPU 수를 `partitions`으로 나눌 수 있는지 확인해야 합니다. 파티션의 결과는 Horovod(데이터 병렬도)에 사용할 모델 복제본의 수로 추론됩니다.

SageMaker 지원 인스턴스 및 비용 정보는 [Amazon SageMaker 요금](https://aws.amazon.com/sagemaker/pricing/)을 참조하세요. 각 인스턴스 유형에 대한 GPU를 조회하려면 [Amazon EC2 인스턴스 유형](https://aws.amazon.com/ec2/instance-types/)을 참조하세요. **Accelerated Computing** 섹션을 사용하여 범용 GPU 인스턴스를 확인하세요. 첨고로, ml.p3.2xlarge는 p3.2xlarge와 동일한 수의 GPU가 있습니다.

### 볼륨 크기 업데이트

`volume_size`에 지정하는 볼륨 크기는 입력 데이터 크기보다 커야 합니다.

### SMP용 매개 변수 사전 설정 및 사용자 정의 mpi 옵션 설정

매개 변수 딕셔너리를 사용하여 마이크로 배치 수(microbatches), 파티션 수, ddp와 함께 데이터 병렬 처리를 사용할지 여부, 파이프라인 전략, 배치 전략 및 기타 BERT 특화 하이퍼 파라미터를 구성할 수 있습니다.

In [ ]:
mpi_options = "-verbose --mca orte_base_help_aggregate 0 "
mpi_options += "--mca btl_vader_single_copy_mechanism none"
smp_parameters = {"optimize": "speed", "microbatches": 12, "partitions": 2, "ddp": True, "pipeline": "interleaved", "overlapping_allreduce": True, "placement_strategy": "cluster", "memory_weight": 0.3}
timeout = 60 * 60
metric_definitions = [{"Name": "base_metric", "Regex": "<><><><><><>"}]

hyperparameters = {"input_dir": "/opt/ml/input/data/train",
                   "output_dir": "./checkpoints", 
                   "config_file": "bert_config.json", 
                   "bert_model": "bert-large-uncased", 
                   "train_batch_size": 48, 
                   "max_seq_length": 128,
                   "max_predictions_per_seq": 20,
                   "max_steps": 7038,
                   "warmup_proportion": 0.2843,
                   "num_steps_per_checkpoint": 200,
                   "learning_rate": 6e-3,
                   "seed": 12439,
                   "steps_this_run": 500,
                   "allreduce_post_accumulation": 1,
                   "allreduce_post_accumulation_fp16": 1,
                   "do_train": 1,
                   "use_sequential": 1,
                   "skip_checkpoint": 1,
                   "smp": 1,
                   "apply_optimizer": 1}

### Instantiate Pytorch Estimator with SMP enabled

In [ ]:
pytorch_estimator = PyTorch("sagemaker_smp_pretrain.py",
                            role=role,
                            instance_type="ml.p3.16xlarge",
                            volume_size=200,
                            instance_count=1,
                            sagemaker_session=sagemaker_session,
                            py_version="py36",
                            framework_version='1.6.0',
                            distribution={
                                "smdistributed": {
                                    "modelparallel": {
                                        "enabled": True,
                                        "parameters": smp_parameters
                                    }
                                },
                                "mpi": {
                                    "enabled": True,
                                    "process_per_host": 8,
                                    "custom_mpi_options": mpi_options,
                                }
                            },
                            source_dir='bert_example',
                            output_path=s3_output_location,
                            max_run=timeout,
                            hyperparameters=hyperparameters,
                            metric_definitions=metric_definitions)

마지막으로 estimator를 사용하여 SageMaker 훈련 작업을 시작합니다.

In [ ]:
pytorch_estimator.fit(data_channels, logs=True)